In [1]:
import numpy as np

In [2]:
# Environment is set up under the assumption that the arrival An is Poisson
class Environment:
    def __init__(self, poisson_lambda, queue_max, service_mu, service_lambda, cost_coeff, initial_queuelength):
        self.poisson_lambda = poisson_lambda
        self.queue_max = queue_max
        self.service_mu = service_mu
        self.service_lambda = service_lambda
        self.cost_coeff = cost_coeff
        self.initial_queuelength = initial_queuelength
        self.queuelength = self.initial_queuelength
    
    def get_current_state(self):
        return self.queuelength
    
    def simulate_and_observe(self, action):
        arrivals = np.random.poisson(self.poisson_lambda)
        self.queuelength = self.queuelength + arrivals
        reward = self.queuelength
        
        if self.queuelength > 20:
            self.queuelength = 20
            
        if self.queuelength > 0:
            service = 0
            if action == 1:
                randp = self.service_mu
            if action == 2:
                randp = self.service_lambda
                    
            if np.random.rand <= randp:
                service = 1
            self.queuelength = self.queuelength - service
        
        reward = reward + self.cost_coeff * action * action;
        
        return [self.queuelength, -reward]

In [3]:
# Every Visit Monte carlo estimate of a policy - action = 1 if state in [0,10], action = 2 if state in [11, 20]
num_episodes = 1000
horizon = 100
state_space = range(21)
action_space = [1,2]
discount_factor = 0.2
epsilon = 0.5 # for epsilon-greedy policy

every_visit_estimate = np.zeros([len(state_space),len(action_space)])
every_visit_count = np.zeros([len(state_space),len(action_space)])

for e in range(num_episodes):
    
    environment = Environment(0.3, 20, 0.2, 0.7, 1, 0)
    state_sequence = []
    action_sequence = []
    reward_sequence = []

    for t in range(horizon):
        state = environment.get_current_state()
        state_sequence.append(state)
        if np.random.rand() <= epsilon:
            action = np.random.choice(action_space)
        else:
            action = action_space[np.argmax(every_visit_estimate[state, :])]
        action_sequence.append(action)
        [state, reward] = environment.simulate_and_observe(action)
        reward_sequence.append(reward)
    # print state_sequence
    # print reward_sequence
    for t, s in enumerate(state_sequence):
        a = action_sequence[t] - 1
        every_visit_return = sum(reward_sequence[t:] * np.power(discount_factor, range(len(state_sequence) - t)))
        every_visit_count[s, a] = every_visit_count[s, a] + 1.0        
        if every_visit_count[s, a] > 0:
            every_visit_estimate[s, a] = ((every_visit_count[s, a] - 1) * every_visit_estimate[s, a] + every_visit_return)/every_visit_count[s, a]            
        # print s, a, t, every_visit_return, every_visit_count[s, a], every_visit_estimate[s, a]
    # for s in state_space:
        # print s, action_space[np.argmax(every_visit_estimate[state, :])]

In [4]:
for s in state_space:
    greedy_action = np.argmax(every_visit_estimate[s, :])
    print s, greedy_action, action_space[greedy_action], every_visit_estimate[s, greedy_action]

0 0 1 -1.90690840445
1 0 1 -3.14107377488
2 0 1 -4.41223898627
3 0 1 -5.69264667624
4 0 1 -6.93726787565
5 0 1 -8.14529367816
6 0 1 -9.4228363752
7 0 1 -10.641277483
8 0 1 -11.9167876661
9 0 1 -13.1746901194
10 0 1 -14.3975645929
11 0 1 -15.62905136
12 0 1 -16.8829421424
13 0 1 -18.1506837614
14 0 1 -19.4152167998
15 0 1 -20.6323283375
16 0 1 -21.908164587
17 0 1 -23.1718703321
18 0 1 -24.3996195122
19 0 1 -25.6089056289
20 0 1 -26.6113692375


In [5]:
every_visit_estimate

array([[ -1.9069084 ,  -4.9186051 ],
       [ -3.14107377,  -6.17212841],
       [ -4.41223899,  -7.42568707],
       [ -5.69264668,  -8.69743906],
       [ -6.93726788,  -9.911483  ],
       [ -8.14529368, -11.1659565 ],
       [ -9.42283638, -12.42098834],
       [-10.64127748, -13.64565759],
       [-11.91678767, -14.94235357],
       [-13.17469012, -16.15682356],
       [-14.39756459, -17.41742532],
       [-15.62905136, -18.67498631],
       [-16.88294214, -19.88284645],
       [-18.15068376, -21.15446616],
       [-19.4152168 , -22.4249251 ],
       [-20.63232834, -23.66727224],
       [-21.90816459, -24.92291477],
       [-23.17187033, -26.12533543],
       [-24.39961951, -27.42329204],
       [-25.60890563, -28.60967151],
       [-26.61136924, -29.60280932]])